In [27]:
%matplotlib inline

from bs4 import BeautifulSoup
import urllib
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pickle
from tqdm import tqdm

import os
import shutil

from descriptor import extract, distance

In [42]:
HOWMANY = 2
MAX_PATCH_SIZE = 64
N_POINTS = 2
DEBUG = False
URL = "https://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/index.html"

In [29]:
def generatePointsForOne(image):
    height,width,_ = image.shape
    height -= MAX_PATCH_SIZE/2
    width -= MAX_PATCH_SIZE/2
    ys = np.random.randint(MAX_PATCH_SIZE/2,height,size=(N_POINTS))
    xs = np.random.randint(MAX_PATCH_SIZE/2,width,size=(N_POINTS))
    return [np.dstack((ys,xs))[0]]

def generatePoints(images):
    points = []
    for image in images:
        points += generatePointsForOne(image)
    return np.array(points)

def getRotats(images,points):
    r = []
    ones = np.ones(shape=(N_POINTS, 1))
    
    print("Generating rotated images")
    for a in tqdm([0,1,2,3,5,10,15,30,60,90,180]):
        for img,pp in zip(images,points):
            COLS,ROWS,_ =  img.shape
            M = cv2.getRotationMatrix2D((COLS/2,ROWS/2),a,1)
            rimg = cv2.warpAffine(img,M,(COLS,ROWS))
            rpp = M.dot(np.hstack([pp, ones]).T).T
            r   += [{"img":rimg,"points":rpp}]
    return r
            

def getScales(images,points):
    r = []
    
    print("Generating scaled images")
    for s in tqdm(np.append(np.arange(0.1,1,0.3),np.arange(2,6))):
        for img,pp in zip(images,points):
            rimg = cv2.resize(img,None,fx=s,fy=s)
            rpp = s*pp
            r   += [{"img":rimg,"points":rpp}]
    return r

def getBlures(images,points):
    r = []
    
    print("Generating blureded images")
    for img,pp in zip(images,points):
        r   += [{"img":img,"points":pp}]
    
    for b in tqdm(np.arange(1,10)):
        for img,pp in zip(images,points):
            rimg = cv2.GaussianBlur(img,(5,5),b)
            r   += [{"img":rimg,"points":pp}]
    return r

def getJpgeds(images,points):
    r = []
    dirname = "tmp"
    label = "file{}_{}.jpeg"
    if os.path.isdir(dirname) and DEBUG:
        shutil.rmtree(dirname) 
    os.mkdir(dirname)
    
    print("Generating images compressed using JPEG algorithm")
    for q in tqdm(np.arange(0,101,10)):
        for i,(img,pp) in enumerate(zip(images,points)):
            filename = os.path.join(dirname, label.format(i,q))
            cv2.imwrite(filename, img, [cv2.IMWRITE_JPEG_QUALITY, i])
            rimg = cv2.imread(filename)
            r   += [{"img":rimg,"points":pp}]
    shutil.rmtree(dirname) 
    return r   

def getTransformations(images,points):
    x = {}
    x["rotat"] = getRotats(images,points)
    x["scale"] = getScales(images,points)
    x["blure"] = getBlures(images,points)
    x["jpged"] = getJpgeds(images,points)
    return x

In [30]:
def getData(t):
    epsilon = 0.05
    if "rotat" in t:
        r0 = 0
        rds = [extract(i["img"],i["points"]) for i in t["rotat"]]
        rd0 = rds[r0]
        rx  = [0,1,2,3,5,10,15,30,60,90,180]
        tmp = np.array([distance(rd0,rd) for rd in rds]).reshape((len(rx),int(len(rds)/len(rx))))
        
        
    
    if "scale" in t:
        pass
    
    if "blure" in t:
        pass
    
    if "jpged" in t:
        pass

def drawPlots(X,Y,L):
    for key in ["rotat","scale","blure","jpged"]:
        if key in X and key in Y and key in L:
            plt.figure()
            plt.xlabel(L[key])
            plt.ylabel("Liczba niepoprawnie określonych odległości w optymistycznem przypadku")
            plt.plot(X[key],Y[key],'go--')

In [31]:
def loadImage(url):
    raw = urllib.request.urlopen(url).read()
    npraw= np.array(bytearray(raw),dtype=np.uint8)
    return cv2.imdecode(npraw,-1)#-1 -> as is (with the alpha channel)

def getImageName(url):
    return url.split('/').pop().split('.').pop(0)

def loadImagesFromSite(url):
    imgs = []
    print("Loading {} image(s) from {}".format(HOWMANY,url))
    with urllib.request.urlopen(url) as response:
        html = BeautifulSoup(response.read(),"lxml")
        for link in tqdm(html.find_all('a')[:HOWMANY]):
            img = loadImage(link.get('href'))
            imgs += [img]
    return np.array(imgs)  

def saveDataset(t):
    for key in ["rotat","scale","blure","jpged"]:
        with open('pickledDataset_{}'.format(key),'wb') as f:
            pickle.dump(t[key],f)
        
def loadDataset():
    try:
        t = {}
        for key in ["rotat","scale","blure","jpged"]:
            with open('pickledDataset_{}'.format(key),'rb') as f:
                t[key] = pickle.load(f)
        return t
    except:
        return None
    
def getDataset():
    t = loadDataset()
    if t is None or DEBUG:
        images = loadImagesFromSite(URL)
        points = generatePoints(images)
        t = getTransformations(images,points)
        
        saveDataset(t)
    return t
    
def display(X,lset,howmany=None):
    if howmany is None:
        howmany = len(X[lset])
    
    print("Preparing images to display")
    for i in tqdm(range(howmany)):
        img = X[lset][i]["img"]
        pp  = X[lset][i]["points"]
        c = (img.max(),img.max(),img.max())
        plt.figure()
        plt.imshow(img)
        
        

In [32]:
# if __name__ =="__main__":
#     t = getDataset()
    

In [41]:
from descriptor import extract, distance
epsilon = 0.05
r0 = 0
rds = []
for i in t["rotat"]:
    print(i["points"])
    rds += [extract(i["img"],i["points"].astype(np.uint64))]
rds = [extract(i["img"],i["points"]) for i in t["rotat"]]
# rd0 = rds[r0]
# rx  = [0,1,2,3,5,10,15,30,60,90,180]
# tmp = np.array([distance(rd0,rd) for rd in rds]).reshape((len(rx),int(len(rds)/len(rx))))

[[  868.   175.]
 [  732.  1238.]]


C:\Users\jurgad\Downloads\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\skimage\filters\_gaussian.py:108: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


TypeError: slice indices must be integers or None or have an __index__ method